# Credit Risk Resampling Techniques

In [26]:
# Import dependencies
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import balanced_accuracy_score, confusion_matrix
from imblearn.metrics import classification_report_imbalanced
from imblearn.over_sampling import RandomOverSampler, SMOTE
from imblearn.under_sampling import ClusterCentroids

import warnings
warnings.filterwarnings('ignore')

# Read the CSV into DataFrame

In [3]:
# Load the data as a DataFrame
lending_df = pd.read_csv(Path("./Resources/lending_data.csv"))
lending_df.head()

,loan_size,interest_rate,homeowner,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status
0,10700.0,7.672,own,52800,0.431818,5,1,22800,low_risk
1,8400.0,6.692,own,43600,0.311927,3,0,13600,low_risk
2,9000.0,6.963,rent,46100,0.349241,3,0,16100,low_risk
3,10700.0,7.664,own,52700,0.430740,5,1,22700,low_risk
4,10800.0,7.698,mortgage,53000,0.433962,5,1,23000,low_risk


In [4]:
le = LabelEncoder()

le.fit(lending_df["homeowner"])
lending_df["homeowner"] = le.transform(lending_df["homeowner"])

lending_df.head()

,loan_size,interest_rate,homeowner,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status
0,10700.0,7.672,1,52800,0.431818,5,1,22800,low_risk
1,8400.0,6.692,1,43600,0.311927,3,0,13600,low_risk
2,9000.0,6.963,2,46100,0.349241,3,0,16100,low_risk
3,10700.0,7.664,1,52700,0.430740,5,1,22700,low_risk
4,10800.0,7.698,0,53000,0.433962,5,1,23000,low_risk


# Split the Data into Training and Testing

In [5]:
# Create our features
X = lending_df.drop(columns = "loan_status")

# Create our target
y = lending_df["loan_status"]

In [6]:
#X.describe()

In [7]:
# Check the balance of our target values
y.value_counts()

low_risk     75036
high_risk     2500
Name: loan_status, dtype: int64

In [8]:
# Create X_train, X_test, y_train, y_test
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)

## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [9]:
# Create the StandardScaler instance
scaler = StandardScaler()

In [10]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset
X_scaler = scaler.fit(X_train)

In [11]:
# Scale the training and testing data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Simple Logistic Regression

In [12]:
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_train_scaled, y_train)

LogisticRegression(random_state=1)

In [13]:
# Calculated the balanced accuracy score
y_pred = model.predict(X_test_scaled)
balanced_accuracy_score(y_test, y_pred)

0.9892813049736127

In [14]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

array([[  615,    10],
       [  102, 18657]], dtype=int64)

In [15]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.86      0.98      0.99      0.92      0.99      0.98       625
   low_risk       1.00      0.99      0.98      1.00      0.99      0.98     18759

avg / total       0.99      0.99      0.98      0.99      0.99      0.98     19384



# Oversampling

In this section, you will compare two oversampling algorithms to determine which algorithm results in the best performance. You will oversample the data using the naive random oversampling algorithm and the SMOTE algorithm. For each algorithm, be sure to complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

### Naive Random Oversampling

In [16]:
# Resample the training data with the RandomOversampler
ros = RandomOverSampler(random_state=1)
X_train_ros, y_train_ros = ros.fit_resample(X_train_scaled, y_train)

# View the count of target classes with Counter
print(Counter(y_train_ros))

Counter({'low_risk': 56277, 'high_risk': 56277})


In [17]:
# Train the Logistic Regression model using the resampled data
ros_model = LogisticRegression(solver = 'lbfgs', random_state = 1)
ros_model.fit(X_train_ros, y_train_ros)

LogisticRegression(random_state=1)

In [18]:
# Calculated the balanced accuracy score
y_pred_ros = ros_model.predict(X_test_scaled)

balanced_accuracy_score(y_test, y_pred_ros)

0.9946414201183431

In [19]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred_ros)

array([[  622,     3],
       [  111, 18648]], dtype=int64)

In [20]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred_ros))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.85      1.00      0.99      0.92      0.99      0.99       625
   low_risk       1.00      0.99      1.00      1.00      0.99      0.99     18759

avg / total       0.99      0.99      1.00      0.99      0.99      0.99     19384



### SMOTE Oversampling

In [21]:
# Resample the training data with SMOTE
X_train_smote, y_train_smote = SMOTE(random_state = 1, sampling_strategy = 1.0).fit_resample(X_train_scaled, y_train)

# View the count of target classes with Counter
Counter(y_train_smote)

Counter({'low_risk': 56277, 'high_risk': 56277})

In [22]:
# Train the Logistic Regression model using the resampled data
smote_model = LogisticRegression(solver = 'lbfgs', random_state = 1)
smote_model.fit(X_train_smote, y_train_smote)

LogisticRegression(random_state=1)

In [23]:
# Calculated the balanced accuracy score
y_pred_smote = smote_model.predict(X_test_scaled)
balanced_accuracy_score(y_test, y_pred_smote)

0.9946414201183431

In [24]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred_smote)

array([[  622,     3],
       [  111, 18648]], dtype=int64)

In [25]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred_smote))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.85      1.00      0.99      0.92      0.99      0.99       625
   low_risk       1.00      0.99      1.00      1.00      0.99      0.99     18759

avg / total       0.99      0.99      1.00      0.99      0.99      0.99     19384



# Undersampling

In this section, you will test an undersampling algorithm to determine which algorithm results in the best performance compared to the oversampling algorithms above. You will undersample the data using the Cluster Centroids algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Display the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [28]:
# Resample the data using the ClusterCentroids resampler
cc = ClusterCentroids(random_state = 1)
X_train_cc, y_train_cc = cc.fit_resample(X_train_scaled, y_train)

# View the count of target classes with Counter
Counter(y_train_cc)

Counter({'high_risk': 1875, 'low_risk': 1875})

In [29]:
# Train the Logistic Regression model using the resampled data
cc_model = LogisticRegression(solver = 'lbfgs', random_state = 1)
cc_model.fit(X_train_cc, y_train_cc)

LogisticRegression(random_state=1)

In [33]:
# Calculate the balanced accuracy score
y_pred_cc = cc_model.predict(X_test_scaled)
balanced_accuracy_score(y_test, y_pred_cc)

0.9932813049736127

In [34]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred_cc)

array([[  620,     5],
       [  102, 18657]], dtype=int64)

In [35]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred_cc))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.86      0.99      0.99      0.92      0.99      0.99       625
   low_risk       1.00      0.99      0.99      1.00      0.99      0.99     18759

avg / total       1.00      0.99      0.99      0.99      0.99      0.99     19384



# Combination (Over and Under) Sampling

In this section, you will test a combination over- and under-sampling algorithm to determine if the algorithm results in the best performance compared to the other sampling algorithms above. You will resample the data using the SMOTEENN algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Display the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [31]:
# Resample the training data with SMOTEENN
# YOUR CODE HERE

# View the count of target classes with Counter
# YOUR CODE HERE

In [32]:
# Train the Logistic Regression model using the resampled data
# YOUR CODE HERE

In [33]:
# Calculate the balanced accuracy score
# YOUR CODE HERE

In [34]:
# Display the confusion matrix
# YOUR CODE HERE

In [35]:
# Print the imbalanced classification report
# YOUR CODE HERE

# Final Questions

1. Which model had the best balanced accuracy score?

   YOUR ANSWER HERE.

2. Which model had the best recall score?

    YOUR ANSWER HERE.

3. Which model had the best geometric mean score?

    YOUR ANSWER HERE.
